In [1]:
#x01_populationStatistics

In [2]:
from chewc.chewc import *

In [3]:
n_chr = 7
n_loci = 1000
founder_pop_size = 2

G = Genome(n_chr, n_loci)
founder_pop = create_pop(G, create_random_pop(G, founder_pop_size))
T = Trait(G, founder_pop, target_mean=0.0, target_variance=1.0)

In [4]:
sim = BreedingSimulation(G, T, h2
                         =0.2, reps=2, pop_size=200, selection_fraction=0.5)


In [5]:
pop_stat_in  = sim.population.haplotypes.sum(dim=1).view((sim.population.size, n_chr* n_loci)).float()
pop_stat = population_statistics(pop_stat_in)

In [8]:
pop_stat['maf'].shape

torch.Size([7000])

In [12]:
for i in pop_stat:
    print(i)

allele_frequencies
genotype_frequencies
heterozygosity
maf
inbreeding_coefficient
ld_matrix
effective_population_size


In [15]:
pop_stat['allele_frequencies'].shape

torch.Size([7000])

In [18]:
pop_stat['genotype_frequencies'][0].shape

torch.Size([200])

In [20]:
pop_stat['genotype_frequencies'][1].shape

torch.Size([200])

In [21]:

pop_stat['heterozygosity'].shape

torch.Size([7000])

In [22]:
pop_stat['maf'].shape

torch.Size([7000])

In [24]:
pop_stat['inbreeding_coefficient'].shape

torch.Size([200])

In [28]:
pop_stat['ld_matrix'].shape

torch.Size([7000, 7000])

In [29]:
pop_stat['effective_population_size']

tensor(161164.8125)

In [35]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MyNetwork(nn.Module):
    def __init__(self, L, P):
        super(MyNetwork, self).__init__()
        
        # Layers for input1 and input3
        self.fc1 = nn.Linear(L, 128)
        self.fc2 = nn.Linear(128, 64)
        
        # Layers for input2 and input4
        self.fc3 = nn.Linear(P, 128)
        self.fc4 = nn.Linear(128, 64)
        
        # Convolutional layers for input_matrix
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        
        # Fully connected layer for input_scalar
        self.fc_scalar = nn.Linear(1, 32)
        
        # Calculate the flattened size of conv layers output
        conv_output_size = L * L * 128
        
        # Final fully connected layers
        self.fc_final1 = nn.Linear(64 * 4 + conv_output_size + 32, 64)
        self.fc_final2 = nn.Linear(64, 2)
        
    def forward(self, input1, input2, input3, input4, input_matrix, input_scalar):
        # Process input1 and input3
        x1 = F.relu(self.fc1(input1))
        x1 = F.relu(self.fc2(x1))
        
        x3 = F.relu(self.fc1(input3))
        x3 = F.relu(self.fc2(x3))
        
        # Process input2 and input4
        x2 = F.relu(self.fc3(input2))
        x2 = F.relu(self.fc4(x2))
        
        x4 = F.relu(self.fc3(input4))
        x4 = F.relu(self.fc4(x4))
        
        # Process input_matrix
        x_matrix = input_matrix.unsqueeze(1)  # Add channel dimension
        x_matrix = F.relu(self.conv1(x_matrix))
        x_matrix = F.relu(self.conv2(x_matrix))
        x_matrix = F.relu(self.conv3(x_matrix))
        x_matrix = torch.flatten(x_matrix, start_dim=1)
        
        # Process input_scalar
        x_scalar = F.relu(self.fc_scalar(input_scalar.unsqueeze(1)))
        
        # Concatenate all features
        x = torch.cat((x1, x2, x3, x4, x_matrix, x_scalar), dim=1)
        
        # Final fully connected layers
        x = F.relu(self.fc_final1(x))
        output = self.fc_final2(x)
        
        return output

# Example usage
batch_size = 32
L = 50
P = 20

model = MyNetwork(L, P)

input1 = torch.randn((batch_size, L))
input2 = torch.randn((batch_size, P))
input3 = torch.randn((batch_size, L))
input4 = torch.randn((batch_size, P))
input_matrix = torch.randn((batch_size, L, L))
input_scalar = torch.randn((batch_size,))

output = model(input1, input2, input3, input4, input_matrix, input_scalar)
print(output.shape)  # Should be (batch_size, 2)


torch.Size([32, 2])
